<a href="https://colab.research.google.com/github/GabrielaCuervoR/AnaliticaMarketing/blob/main/b_exploraci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import a_funciones as fn
from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
##### conectarse a BD #######
conn= sql.connect('data\\db_movies2')
cur=conn.cursor()

In [ ]:
### para ver las tablas que hay en la base de datos
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('rating_user2',),
 ('rating_final',),
 ('fecha_nueva',),
 ('genres4',),
 ('movies3',),
 ('movies4',),
 ('ratings1',),
 ('ratingtitle',)]

In [ ]:
########### traer tabla de BD a python ####
movies= pd.read_sql("""select *  from movies""", conn)
ratings = pd.read_sql('select * from ratings', conn)

In [ ]:
#####Exploración inicial #####

### Identificar campos de cruce y verificar que estén en mismo formato ####
### verificar duplicados

movies.info()
movies.head()
movies.duplicated().sum()

ratings.info()
ratings.head()
ratings.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


0

Exploración de base de datos


In [ ]:
##### Descripción base de ratings
###calcular la distribución de calificaciones
cali=pd.read_sql(""" select
                          "rating" as calificación,
                          count(*) as cantidad
                          from ratings
                          group by "rating"
                          order by cantidad desc""", conn)

In [ ]:
cali ##Verificar tabla de distribución calificaciones

,calificación,cantidad
0,4.0,26818
1,3.0,20047
2,5.0,13211
3,3.5,13136
4,4.5,8551
5,2.0,7551
6,2.5,5550
7,1.0,2811
8,1.5,1791
9,0.5,1370


In [ ]:
##Grafico de las distribuciones
data  = go.Bar( x= cali.calificación,y=cali.cantidad, text=cali.cantidad, textposition="outside")
Layout=go.Layout(title="Conteo de calificaciones",xaxis={'title': 'Calificación', 'tickvals': cali.calificación},yaxis={'title':'Conteo'})
go.Figure(data,Layout)

In [ ]:
### calcular cada usuario cuantas peliculas calificó
rating_users=pd.read_sql(''' select "userId" as user_id,
                         count(*) as calificación
                         from ratings
                         group by "user_id"
                         order by calificación asc
                         ''',conn )

In [ ]:
rating_users##Verificar tabla

,user_id,calificación
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
605,274,1346
606,448,1864
607,474,2108
608,599,2478


In [ ]:
fig  = px.histogram(rating_users, x= 'calificación', title= 'Hist frecuencia de número de calificaciones por usario')
fig.show()

In [ ]:
## Crear tabla con usuarios que hayan calificado más de 20 peliculas y menos de 600 para obtener una mejor distribución
rating_users2=pd.read_sql(''' select "userId" as userId,
                         count(*) as calificación
                         from ratings
                         group by "userId"
                         having calificación >=20 and calificación <=600
                         order by calificación asc
                         ''',conn )

In [ ]:
rating_users2 ##Verificar tabla

,userId,calificación
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
573,219,528
574,28,570
575,91,575
576,555,578


In [ ]:
fig  = px.histogram(rating_users2, x= 'calificación', title= 'Hist frecuencia de número de calificaciones por usario')
fig.show()

In [ ]:
## Crear tabla de acuerdo a la calificación de cada pelicula, para observar el top 10 de las más calificadas
rating_movies=pd.read_sql(''' select movieId ,
                         count(*) as calificación
                         from ratings
                         group by "movieId"
                         order by calificación desc limit 10
                         ''',conn )


In [ ]:
rating_movies

,movieId,calificación
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
5,260,251
6,480,238
7,110,237
8,589,224
9,527,220


In [ ]:
rating_movies = rating_movies.astype({'movieId': 'str'}) ## Se transdorma a str para poder ver la distribución en gráfica

In [ ]:
rating_movies.info() ##Verificar el cambio

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieId       10 non-null     object
 1   calificación  10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


In [ ]:
data  = go.Bar( x= rating_movies.movieId,y=rating_movies.calificación, text=rating_movies.movieId, textposition="outside")
Layout=go.Layout(title="Conteo de calificaciones",xaxis={'title': 'movieId'},yaxis={'title':'Conteo'})
go.Figure(data,Layout)

In [ ]:
fn.ejecutar_sql('preprocesamientomovies.sql', cur)

In [ ]:
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('rating_user2',),
 ('rating_final',),
 ('fecha_nueva',)]

Preprocesamiento base de datos

In [ ]:
##Separación de los generos para poder ejecutar la tabla de movies de una mejor manera.
movies=pd.read_sql("""select * from movies""", conn)
genres=movies['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)

In [ ]:
genres= genres.drop(['(no genres listed)'], axis=1) # Se borra la columna no genres listed, porque no da información

In [ ]:
genres4= genres.copy() ##Copia de la tabla para no hacer cambios que modifiquen la tabla

In [ ]:
##### Convertir los argumentos true y false en 1 y 0, para poder trabajar mejor.
genres4 = genres4.replace({True: 1, False: 0})

In [ ]:
genres4 = genres4.astype(int) ## Se convierte a entero para que de mejor información

In [ ]:
genres4.info() ##Información de la tabla de generos de la pelicula

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Action       9742 non-null   int64
 1   Adventure    9742 non-null   int64
 2   Animation    9742 non-null   int64
 3   Children     9742 non-null   int64
 4   Comedy       9742 non-null   int64
 5   Crime        9742 non-null   int64
 6   Documentary  9742 non-null   int64
 7   Drama        9742 non-null   int64
 8   Fantasy      9742 non-null   int64
 9   Film-Noir    9742 non-null   int64
 10  Horror       9742 non-null   int64
 11  IMAX         9742 non-null   int64
 12  Musical      9742 non-null   int64
 13  Mystery      9742 non-null   int64
 14  Romance      9742 non-null   int64
 15  Sci-Fi       9742 non-null   int64
 16  Thriller     9742 non-null   int64
 17  War          9742 non-null   int64
 18  Western      9742 non-null   int64
dtypes: int64(19)
memory usage: 1.4 MB


In [ ]:
##Insertar la columna de movieId y el title
genres4.insert(0, 'movieId', movies['movieId'])
genres4.insert(1, 'title', movies['title'])

In [ ]:
genres4.to_sql('genres4',conn, if_exists='replace' ) ##Se manda la tabla a la base de datos

9742

In [ ]:
cur.execute("ALTER TABLE genres4 ADD COLUMN fecha INTEGER") #Se agrega la columna fecha

In [ ]:
## Se extrae el año del titulo, que quiere decir cuando fue el estreno de la pelicula
cur.execute("""
    UPDATE genres4
    SET fecha = CAST(SUBSTR(title, -5, 4) AS INTEGER)
""")

In [ ]:
movies3 = pd.read_sql(""" select * from genres4""", conn)

In [ ]:
movies3.to_sql('movies3',conn, if_exists='replace' )

9742

In [ ]:
## Se escala la fecha del año
sc=MinMaxScaler()
movies3[["year_sc"]]=sc.fit_transform(movies3[['fecha']])

In [ ]:
movies4= movies3.drop(columns=['title','index','fecha']) ## Se elimina la fecha y el title a la tabla movies3 y se guarda en movies 4

In [ ]:
movies4.to_sql('movies4',conn, if_exists='replace' )

9742

In [ ]:
joblib.dump(movies4,"salidas\\movies4.joblib") ### para utilizar en segundos modelos

['salidas\\movies4.joblib']

In [ ]:
##Se usa la tabla de rating_final para poder hacer los modelos
rating_final= pd.read_sql('select * from rating_final', conn)

In [ ]:
# ##Convertir columna timestamp a formato fecha
rating_final['fecha']= pd.read_sql("""SELECT fecha FROM fecha_nueva""", conn)

In [ ]:
##Se hace una copia de rating_final
ratings1= rating_final.copy()

In [ ]:
##Se elimina el timestamp de la tabla de ratings, para no afectar los sistemas de recomendación
ratings1= ratings1.drop(['timestamp'],axis=1)

In [ ]:
ratings1.to_sql('ratings1',conn, if_exists='replace' )

66702

In [ ]:
##Se crea la tabla ratingtitle para la realización de modelos
ratingtitle= pd.read_sql("""select b.*,a.title
            from ratings1 b left join movies a on b.movieId=a.movieId
            """, conn)

In [ ]:
ratingtitle =ratingtitle.drop(['index'], axis=1)#Se elimina la columna index

In [ ]:
ratingtitle.to_sql('ratingtitle',conn, if_exists='replace' ) #Se agrega a sql

66702